In [1]:
import ast
import re
import pandas as pd
from datasets import load_dataset

In [2]:
cocktails_dataset = load_dataset("erwanlc/cocktails_recipe")

Using custom data configuration erwanlc--cocktails_recipe-6ed27d67a557fd3e
Reusing dataset csv (C:\Users\lecover\.cache\huggingface\datasets\csv\erwanlc--cocktails_recipe-6ed27d67a557fd3e\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
100%|██████████| 1/1 [00:00<00:00, 27.46it/s]


In [3]:
cocktails_dataset = cocktails_dataset["train"].to_pandas()

In [4]:
cocktails_dataset["ingredients"] = cocktails_dataset["ingredients"].map(lambda x: ast.literal_eval(x))

In [8]:
cocktails_dataset.head()

,title,glass,garnish,recipe,ingredients,raw_ingredients
0,Abacaxi Ricaço,Pineapple shell (frozen) glass,Cut a straw sized hole in the top of the pinea...,Cut the top off a small pineapple and carefull...,"[[1 whole, Pineapple (fresh)], [9 cl, Havana C...",[
1,Abbey,Coupe glass,Orange zest twist,SHAKE all ingredients with ice and fine strain...,"[[4.5 cl, Rutte Dry Gin], [2.25 cl, Lillet Bla...",[
2,A.B.C. Cocktail,Nick & Nora glass,Lemon zest twist & Luxardo Maraschino cherry,TEAR mint and place in shaker. Add other ingre...,"[[7 fresh, Mint leaves], [3 cl, Tawny port], [...",[
3,Absinthe Cocktail,Coupe glass,Mint leaf,SHAKE all ingredients with ice and fine strain...,"[[3 cl, La Fée Parisienne absinthe], [7.5 cl, ...",[
4,Absinthe Frappé,Old-fashioned glass,Mint sprig,SHAKE all ingredients with ice and fine strain...,"[[4.5 cl, La Fée Parisienne absinthe], [1.5 cl...",[


In [11]:
import json

with open('ingredients_mapping.txt', 'r') as f:
    ingr_mapping = json.load(f)

In [33]:
def get_raw_ingredients(ingredients):
    return [item[1] for item in ingredients]
def convert_alcohol(ingredients, alcohol):
    return [alcohol if alcohol in item else item for item in ingredients]
def map_ingredients(ingredients_list):
    return list(map(ingr_mapping.get, ingredients_list, ingredients_list))
alcohol_list = ["gin", "vodka", "cognac", "triple sec", "bourbon whiskey", "absinthe", "tequila", "angostura", "scotch whisky", "cherry brandy liqueur", "amaretto", "limoncello", "dry vermouth", "grand marnier", "rye whiskey", "suze", "prosecco", "mezcal", "campari", "cachaça", "irish cream", "orange curaçao", "blue curaçao" "galliano", "falernum liqueur"]
cocktails_dataset["raw_ingredients"] = cocktails_dataset["ingredients"].map(lambda ingredients: get_raw_ingredients(ingredients))
cocktails_dataset["raw_ingredients"] = cocktails_dataset["raw_ingredients"].map(lambda ingredients: [re.sub(r"giffard|monin|thomas henry","", ingredient).strip().lower() for ingredient in ingredients])
cocktails_dataset["raw_ingredients"] = cocktails_dataset["raw_ingredients"].map(lambda ingredients: [re.sub(r"\([^()]*\)","", ingredient).strip().lower() for ingredient in ingredients])
for alcohol in alcohol_list:
    cocktails_dataset["raw_ingredients"] = cocktails_dataset["raw_ingredients"].map(lambda ingredients: convert_alcohol(ingredients, alcohol))
cocktails_dataset["raw_ingredients"] = cocktails_dataset["raw_ingredients"].map(lambda ingredients: map_ingredients(ingredients))

In [70]:
def concat_new_old(x):
    new_col = list(zip(x["ingredients"], x["raw_ingredients"]))
    return [[raw[0], new] for raw, new in new_col]
cocktails_dataset["raw_ingredients"] = cocktails_dataset.apply(lambda x: concat_new_old(x), axis=1)

In [ ]:
cocktails_dataset.to_csv("./train.csv", index=False)

In [22]:
raw_ingredients = cocktails_dataset["raw_ingredients"].tolist()

In [23]:
from itertools import chain

raw_ingredients = list(chain.from_iterable(raw_ingredients))

In [24]:
from collections import Counter
raw_ingredients = Counter(raw_ingredients)
raw_ingredients.most_common()

[('gin', 2061),
 ('lemon juice', 1698),
 ('lime juice', 1629),
 ('sugar syrup', 1577),
 ('angostura', 1183),
 ('vodka', 1126),
 ('white rum', 821),
 ('cognac', 749),
 ('triple sec', 548),
 ('dry vermouth', 518),
 ('red vermouth', 500),
 ('orange juice', 496),
 ('tequila', 466),
 ('egg white', 463),
 ('bourbon whiskey', 452),
 ('pineapple juice', 440),
 ('chilled water', 397),
 ('scotch whisky', 370),
 ('soda water', 331),
 ('absinthe', 291),
 ('grapefruit juice', 282),
 ('maraschino', 272),
 ('amber rum', 269),
 ('apple juice', 267),
 ('giffard grenadine syrup', 251),
 ('cranberry juice', 239),
 ('st-germain elderflower liqueur', 239),
 ('italian red bitter liqueur', 234),
 ('cherry brandy liqueur', 234),
 ('single cream / half-and-half', 231),
 ('mint leaves', 223),
 ('rye whiskey', 196),
 ('amaretto', 195),
 ('green chartreuse liqueur', 186),
 ('brut champagne', 185),
 ('*special ingredient #1', 183),
 ('coffee liqueur', 171),
 ("peychaud's bitter", 169),
 ('mezcal', 161),
 ('bénédic

In [95]:
to_save = raw_ingredients.most_common()

In [96]:
to_save = [item[0] for item in to_save]

In [101]:
textfile = open("ingredients_mapping.txt", "w", encoding="utf-8")
textfile.write("{")
for element in to_save:
    textfile.write("\""+element + "\": " + ",\n")
textfile.write("}")
textfile.close()